### Load Business AR Data - Cohort 7

This will gather data, create the nanoid and load data into the Business AR database

## Setup

In [2]:
%load_ext sql
%load_ext dotenv
%config SqlMagic.named_parameters="enabled" 

In [ ]:
import pandas as pd
import numpy as np
from dotenv import find_dotenv
from dotenv import load_dotenv
from nanoid import generate
load_dotenv(find_dotenv())

In [4]:
import string
nanoid_charset = string.ascii_letters + string.digits

In [5]:
import google.auth
credentials, project_id = google.auth.default()

In [6]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

# def get_pool():
#     # create connection pool with 'creator' argument to our connection object function
#     pool = sqlalchemy.create_engine(
#         "postgresql+pg8000://",
#         creator=get_conn,
#     )
    
#     return pool

## Reset Environment Variables

In [7]:
%dotenv -o

In [ ]:
def get_pool(db_user, db_password, db_name, instance_connection_name):
    # function to return the database connection object
    def get_conn():
        conn = connector.connect(
            instance_connection_name,
            "pg8000",
            user=db_user,
            password=db_password,
            db=db_name
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
        echo=True,
    )
    return pool

## Create Business Connection
business_pool = get_pool(
    db_user=os.getenv("BUSINESS_USERNAME", ""),
    db_password=os.getenv("BUSINESS_PASSWORD", ""),
    db_name=os.getenv("BUSINESS_NAME", ""),
    instance_connection_name=os.getenv("BUSINESS_CONNECTION", "")
)
%sql business_pool --alias business

## Create Warehouse Connection
warehouse_pool = get_pool(
    db_user=os.getenv("WAREHOUSE_USERNAME", ""),
    db_password=os.getenv("WAREHOUSE_PASSWORD", ""),
    db_name=os.getenv("WAREHOUSE_NAME", ""),
    instance_connection_name=os.getenv("WAREHOUSE_CONNECTION", "")
)
%sql warehouse_pool --alias warehouse


## Create Business Connection

In [ ]:
# DB_USER = os.getenv("BUSINESS_USERNAME", "")
# DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
# DB_NAME = os.getenv("BUSINESS_NAME", "")
# INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

# business_pool = get_pool()
# %sql business_pool --alias business
%sql business

In [ ]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

## List Database Connections

In [ ]:
%sql --connections

## Query Business Database

In [14]:
%sql business

In [ ]:
%%sql business_data <<
select * from business;

In [ ]:
business_data

## Create Warehouse Connection

In [13]:
# DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
# DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
# DB_NAME = os.getenv("WAREHOUSE_NAME", "")
# INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

# warehouse_pool = get_pool()
# %sql warehouse_pool --alias warehouse
# %sql warehouse

In [ ]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

## Query Warehouse Database

In [ ]:
%sql warehouse

In [ ]:
%%sql colin_data <<
SELECT co.corp_num
     , co.recognition_dts
     , EXTRACT(YEAR FROM co.last_ar_filed_dt) AS last_ar_filed_year
     , co.admin_email
     , cn.CORP_NME
     , ct.corp_class
FROM "colin"."corporation"   co
   , "colin".corp_type       ct
   , "colin".corp_state      cs
   , "colin".corp_name       cn
WHERE co.corp_typ_cd    = ct.corp_typ_cd
  AND co.corp_num       = cs.corp_num
  AND co.corp_num       = cn.corp_num
  AND cs.end_event_id   IS NULL
  and cn.end_event_id is null
  and cn.corp_name_typ_cd = 'CO'
  AND cs.state_typ_cd   = 'ACT'                                                                                -- active
  AND ct.corp_class     = 'BC'                                                                                 -- BC Corporations
  AND co.corp_typ_cd   <> 'BEN'                                                                                -- no Benefit Companies
  AND co.admin_email IS NOT NULL                                                                               -- they have an email
  AND co.send_ar_ind = 'Y'                                                                                     -- AR reminder indicator is "Y"
  AND NOT EXISTS (SELECT 'x'
                  FROM "colin".filing f, "colin".event e, "colin".filing_user u
                  WHERE f.event_id = e.event_id
                    AND f.event_id = u.event_id                                                                -- no previous BCOL filings
                    AND e.corp_num = co.corp_num
                    AND u.role_typ_cd = 'bcol')
  AND NOT EXISTS (SELECT 'x'
                  FROM "colin".corporation
                  WHERE admin_email = co.admin_email
                    AND corp_num <> co.corp_num)                                                               -- no other business using the same email
  AND (date_part('doy', co.recognition_dts) BETWEEN date_part('doy', current_date)                            -- AR reminder within the next 14 days based on the day of year
       AND date_part('doy', current_date))                                      -- AR reminder on the anniversary date
  AND (
       -- Exclude companies founded in the current year, include those from previous year
       (EXTRACT(YEAR FROM co.recognition_dts) = EXTRACT(YEAR FROM current_date) - 1 AND co.last_ar_filed_dt IS NULL)
       -- Or include if last_ar_filed_dt is not NULL and was filed in the previous year
       OR (co.last_ar_filed_dt IS NOT NULL AND EXTRACT(YEAR FROM co.last_ar_filed_dt) = EXTRACT(YEAR FROM current_date) - 1)
      );

In [ ]:
colin_df = colin_data.DataFrame()
colin_df

In [ ]:
# Print original columns to verify names
print("Original columns:", colin_df.columns.tolist())

# Rename columns (adjusted based on actual column names)
colin_df.rename(columns={
    'corp_num': 'identifier',
    'corp_nme': 'legal_name',  # Adjusted to match actual column name
    'recognition_dts': 'founding_date',
    'admin_email': 'email',
    'last_ar_filed_year': 'last_ar_reminder_year',
    'corp_class': 'legal_type'  # Use 'corp_class' as 'legal_type'
}, inplace=True)

# Print columns after renaming to confirm
print("Columns after renaming:", colin_df.columns.tolist())

# Replace 'None' strings with actual None values
colin_df.replace('None', None, inplace=True)

# Convert data types
colin_df['last_ar_reminder_year'] = pd.to_numeric(colin_df['last_ar_reminder_year'], errors='coerce').astype('Int64')
colin_df['founding_date'] = pd.to_datetime(colin_df['founding_date'], errors='coerce')

# Select the required columns
colin_df = colin_df[['identifier', 'legal_name', 'legal_type', 'founding_date', 'last_ar_reminder_year', 'email']]

# Function to escape and format values for SQL
def escape_and_format(value):
    if isinstance(value, str):
        # Escape single quotes in strings
        return "'" + value.replace("'", "''") + "'"
    elif pd.isnull(value):
        # Handle None values
        return 'NULL'
    elif isinstance(value, pd.Timestamp):
        # Format dates as 'YYYY-MM-DD'
        return "'" + value.strftime('%Y-%m-%d') + "'"
    else:
        # For other data types, convert to string or handle None
        return str(int(value)) if value is not None else 'NULL'

# Generate the VALUES part of the INSERT statement
values = ",\n".join([
    "(" + ", ".join([
        escape_and_format(row['identifier']),
        escape_and_format(row['legal_name']),
        escape_and_format(row['legal_type']),
        escape_and_format(row['founding_date']),
        escape_and_format(row['last_ar_reminder_year']),
        escape_and_format(row['email'])
    ]) + ")"
    for _, row in colin_df.iterrows()
])

# Complete the INSERT statement
insert_statement = f"""INSERT INTO "public"."business" (identifier, legal_name, legal_type, founding_date, last_ar_reminder_year, email)
VALUES 
{values};"""

# Print the INSERT statement
print(insert_statement)


In [ ]:
# Verify connection by querying the current database
%sql business


In [ ]:
%%sql
SELECT * from business;

In [ ]:
try:
    colin_df.to_sql(name='business', con=business_pool, if_exists='append', index=False)
except Exception as e:
    print("An error occurred during to_sql operation:", e)


In [82]:
# Close the connection and reconnect
business_pool.dispose()  # Disposes the current pool and closes all connections
business_pool = get_pool()  # Recreate the connection pool


In [ ]:
colin_df

## Load from CSV

In [ ]:
colin_df = pd.read_csv('cohort_7.csv') 
colin_df

In [ ]:
# colin_df["identifier"] = colin_df["Jurisdiction"] + colin_df['corp_num'].apply('{:0>7}'.format)
# colin_df['nano_id'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
# colin_df['tax_id'] = None
# colin_df['id'] = np.arange(1, colin_df.shape[0] + 1) + 7
#to_bar_df = pd.DataFrame()
#to_bar_df[['id','legal_name','legal_type','identifier','tax_id','nano_id']] = colin_df[['id','Name','Jurisdiction','identifier','tax_id','nano_id']]
#to_bar_df


In [ ]:
# colin_df['tax_id'] = None
# colin_df['ar_reminder_flag'] = None
# colin_df['state'] = None
# colin_df['op_state'] = None
# colin_df['corp_class'] = None
to_bar_df = colin_df

In [ ]:
to_bar_df.to_sql(name='b2', con=business_pool, if_exists='append', index=False)

In [ ]:
%%sql
INSERT INTO business (id, legal_name, legal_type, identifier, email, founding_date, last_ar_reminder_year, state)
SELECT 
    (SELECT COALESCE(MAX(id), 0) FROM business) + ROW_NUMBER() OVER (), 
    legal_name, 
    legal_type, 
    identifier, 
    email, 
    founding_date::date, 
    last_ar_reminder_year, 
    'ACT'
FROM b2;


In [ ]:
to_inv_email_df = pd.DataFrame()
to_inv_email_df[['id','recipients']] = colin_df[['id','email']]
to_inv_email_df['business_id'] = to_inv_email_df['id']
to_inv_email_df['token'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
to_inv_email_df['message'] = colin_df.apply(lambda row: 'AR Invitation', axis = 1)
to_inv_email_df['status'] = colin_df.apply(lambda row: 'SENT', axis = 1)
to_inv_email_df

In [ ]:
%%sql business_data <<
select * from business where id = 9

## If using csv files then no need to run following two cells

In [ ]:
business_df = business_data.DataFrame()

In [ ]:
to_inv_email_df = pd.DataFrame()
to_inv_email_df[['id','recipients']] = business_df[['id','email']]
to_inv_email_df['business_id'] = to_inv_email_df['id']
to_inv_email_df['token'] = business_df.apply(lambda row: generate(nanoid_charset), axis = 1)
to_inv_email_df['message'] = business_df.apply(lambda row: 'AR Invitation', axis = 1)
to_inv_email_df['status'] = business_df.apply(lambda row: 'SENT', axis = 1)
to_inv_email_df

Writes into inv_ch6 table

In [ ]:
to_inv_email_df.to_sql(name='inv_ch6', con=business_pool, if_exists='append', index=False)

Insert into invitations table from inv_ch6 table

In [ ]:
%%sql
insert into invitations (id,recipients,message,sent_date,token,status,business_id)
(select nextval('invitations_id_seq'::regclass), recipients, message, now(), token, 'SENT', business_id from inv_ch6)

## Output to CSV

In [ ]:
%%sql business_data <<
select b.identifier, b.legal_name, b.legal_type, i.token, i.recipients from business b, invitations i where b.id=i.business_id and b.id >= 104

In [ ]:
business_data

In [ ]:
bdf = business_data.DataFrame()
# bdf = bdf[['legal_name','legal_type','identifier','nano_id']]
base_url='https://annualreport.business.bcregistry.gov.bc.ca/en-CA'
bdf['url'] = f'{base_url}?nanoid=' + bdf['token']
bdf

In [ ]:
import time
from datetime import datetime
from datetime import timezone

time_stamp = time.time()
now = datetime.utcfromtimestamp(time_stamp).replace(tzinfo=timezone.utc)
# local_now = now.astimezone(Pacific)
local_now = now.astimezone()
local_now.strftime("%Y.%m.%d.%H")

In [ ]:
with open('business-ar.'+local_now.strftime("%Y.%m.%d.%H")+'.csv', 'a') as f:      
    f.write('\n\n Business Annual Report\n')
    bdf.to_csv(f, sep=',', encoding='utf-8', index=False)    

## Set Invitations

In [ ]:
%%sql
INSERT INTO invitations (id, recipients, message, sent_date, token, status, additional_message, business_id)
SELECT 
    nextval('invitations_id_seq'::regclass), 
    'test@example.com', 
    'Annual Report Due', 
    now(), 
    b2.nano_id,
    'SENT', 
    NULL, 
    b.id
FROM 
    business b
JOIN 
    b2 ON b.id = b2.id;
